In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import mean_absolute_error as MAE,mean_squared_error as MSE,r2_score as R2
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
import joblib


In [ ]:
data=pd.read_csv("/content/mission1_data (1).csv")
data

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data["Test Preparation Course"].fillna("Nat complete",inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
data.head()

In [ ]:
data["Exam_score"]=data["Math Score"]+data["Reading Score"]+data["Writing Score"]

In [ ]:
data.head()

In [ ]:
data.drop(["Math Score","Reading Score","Writing Score"],axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
categorical_cols = ["Gender", "Parental Education Level", "Lunch Type", "Test Preparation Course"]

le_dict = {}

# أنشئ LabelEncoder لكل عمود وخزّنه في القاموس
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    le_dict[col] = le

# خزّن قاموس الـ LabelEncoders
joblib.dump(le_dict, "/content/le.pkl")

In [ ]:
min=MinMaxScaler()
data[['Study Time', 'Absences']]=min.fit_transform(data[['Study Time', 'Absences']])
joblib.dump(min,"min.pkl")

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
x=data.drop('Exam_score',axis=1)
y=data['Exam_score']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [ ]:
dt=DecisionTreeRegressor()

rf=RandomForestRegressor()
lr=LinearRegression()

In [ ]:
dt.fit(x_train,y_train)

In [ ]:
rf.fit(x_train,y_train)

In [ ]:
lr.fit(x_train,y_train)

In [ ]:
y_pred_dt=dt.predict(x_test)
y_pred_rf=rf.predict(x_test)
y_pred_lr=lr.predict(x_test)

In [ ]:
print(MSE(y_test,y_pred_lr))
print(MAE(y_test,y_pred_lr))
print(R2(y_test,y_pred_lr))

In [ ]:
print(MSE(y_test,y_pred_dt))
print(MAE(y_test,y_pred_dt))
print(R2(y_test,y_pred_dt))

In [ ]:
print(MSE(y_test,y_pred_rf))
print(MAE(y_test,y_pred_rf))
print(R2(y_test,y_pred_rf))

In [ ]:
joblib.dump(rf,"rf.pkl")

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x=y_test,y=y_pred_dt,color='red',label='dt model')
sns.scatterplot(x=y_test,y=y_pred_rf,color='green',label='rf model')
sns.scatterplot(x=y_test,y=y_pred_lr,color='blue',label='lr model')
plt.title("y_test vs y_pred")
plt.xlabel('y_test')
plt.ylabel('y_pred')
plt.show()

In [ ]:
data.head(2)

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(x=data['Exam_score'],color='blue',bins=20,kde=True)
plt.xlabel('Exam_score')
plt.ylabel('frequency')
plt.title("relation between target and feature")
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
sns.scatterplot(x=data['Exam_score'],y=data['Study Time'],color='black')
plt.xlabel('Exam_score')
plt.ylabel('Study Time')
plt.title("relation between two column")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x=data["Parental Education Level"],y=data["Exam_score"],color='brown')
plt.xlabel("Parental Education Level")
plt.ylabel("Exam_score")
plt.title("impact Exam score on PEL")
plt.show()


In [ ]:
params={'max_depth':[5,15,25]}
grid_mod=GridSearchCV(DecisionTreeRegressor(),params,cv=5)
grid_mod.fit(x_train,y_train)
y_pred_grid=grid_mod.best_estimator_.predict(x_test)
print(grid_mod.best_params_)
print(MSE(y_test,y_pred_grid))

In [ ]:
par={'max_depth':[66,77,88]}
rand_mod=RandomizedSearchCV(RandomForestRegressor(),par,cv=5)
rand_mod.fit(x_train,y_train)
print(rand_mod.best_params_)
y_pred_rand=rand_mod.best_estimator_.predict(x_test)
print(MSE(y_test,y_pred_rand))

In [ ]:
import gradio as gr

In [ ]:
data.info()

In [ ]:
data["Parental Education Level"].value_counts()

In [ ]:
le=joblib.load("/content/le.pkl")
min=joblib.load("/content/min.pkl")
rf=joblib.load("/content/rf.pkl")
import pandas as pd
import gradio as gr




def haneen(Gender, PEL, LT, TPC, ST, Absences):
  try:
    inputs_data=pd.DataFrame({
        "Gender":[Gender],
        "Parental Education Level":[PEL],
        "Lunch Type":[LT],
        "Test Preparation Course":[TPC],
        "Study Time":[ST],
        "Absences":[Absences]
    })
    for i in inputs_data:
      if inputs_data[i].dtype=='object':
        inputs_data[i]=le.transform(inputs_data[i])
    inputs_data[["Study Time","Absences"]]=min.transform(inputs_data[["Study Time","Absences"]])
    prediction=rf.predict(inputs_data)
    print(prediction)
    print(prediction)
    return prediction[0]
  except Exception as e:
    return str(e)
gr.Interface(
    fn=haneen,
    inputs=[
    gr.Dropdown(label='Gender',choices=["female","male"]),
    gr.Dropdown(label='Parental Education Level',choices=["High School","Bachelor","Associate","Master","Some College"]),
    gr.Dropdown(label='Lunch Type',choices=["Standard","Free/Reduced"]),
    gr.Dropdown(label="Test Preparation Course",choices=["Completed","Nat complete"]),
    gr.Number(label="Study Time"),
    gr.Number(label="Absences")
        ],
    outputs=gr.Text("prediction")

).launch()




In [ ]:
le = joblib.load("/content/le.pkl")  # قاموس من LabelEncoders
min = joblib.load("/content/min.pkl")
rf = joblib.load("/content/rf.pkl")

import pandas as pd
import gradio as gr

def haneen(Gender, PEL, LT, TPC, ST, Absences):
  try:
    inputs_data = pd.DataFrame({
        "Gender": [Gender],
        "Parental Education Level": [PEL],
        "Lunch Type": [LT],
        "Test Preparation Course": [TPC],
        "Study Time": [ST],
        "Absences": [Absences]
    })

    # التحويل باستخدام القاموس
    for col in ["Gender", "Parental Education Level", "Lunch Type", "Test Preparation Course"]:
        inputs_data[col] = le[col].transform(inputs_data[col])

    # التحويل للأرقام باستخدام MinMaxScaler
    inputs_data[["Study Time", "Absences"]] = min.transform(inputs_data[["Study Time", "Absences"]])

    prediction = rf.predict(inputs_data)
    return prediction[0]
  except Exception as e:
    return str(e)

# Gradio واجهة
gr.Interface(
    fn=haneen,
    inputs=[
        gr.Dropdown(label='Gender', choices=["female", "male"]),
        gr.Dropdown(label='Parental Education Level', choices=["High School", "Bachelor", "Associate", "Master", "Some College"]),
        gr.Dropdown(label='Lunch Type', choices=["Standard", "Free/Reduced"]),
        gr.Dropdown(label="Test Preparation Course", choices=["Completed", "Nat complete"]),
        gr.Number(label="Study Time"),
        gr.Number(label="Absences")
    ],
    outputs=gr.Text(label="Prediction")
).launch()
